In [1]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import f1_score, accuracy_score
from tqdm import tqdm
from PIL import Image

In [1]:
import os
import time
import csv
from PIL import Image
from tqdm import tqdm 
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset, Subset, random_split
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from torchsummary import summary
from torchvision.transforms.v2 import ToDtype
from torchvision.transforms import Normalize
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import torchvision.transforms as transforms
from torchvision.transforms import RandomHorizontalFlip, RandomRotation, RandomVerticalFlip, ColorJitter, ToTensor, Normalize

In [10]:
# Устройство для вычислений
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Пути к данным
source_directory = r'D:\data sets\ml-intensive-yandex-academy-autumn-2024\human_poses_data'
csv_train_ans = r'D:\data sets\ml-intensive-yandex-academy-autumn-2024\human_poses_data\train_answers.csv'
root_dir_train = r'img_train'
root_dir_test = r'D:\data sets\ml-intensive-yandex-academy-autumn-2024\human_poses_data\img_test'
activity_categories_file = r'activity_categories.csv'

# Гиперпараметры
batch_size = 128
learning_rate = 0.001
num_classes = 10  # Укажите количество классов в задаче


# Класс для загрузки тестового датасета
class CustomTrainDataset(Dataset):
    def __init__(self, root_dir, annotations_file, transform=None):
        self.root_dir = root_dir
        self.annotations = pd.read_csv(annotations_file)
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        # Преобразование значения в строку
        img_name = str(self.annotations.iloc[idx, 0])  # Преобразуем в строку
        img_path = os.path.join(self.root_dir, img_name)
        image = Image.open(img_path).convert("RGB")
        label = int(self.annotations.iloc[idx, 1])  # Убедитесь, что метка - это целое число

        if self.transform:
            image = self.transform(image)

        return image, label

# Определение трансформаций
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Загрузка данных
train_dataset = CustomTrainDataset(annotations_file=csv_train_ans, root_dir=root_dir_train, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Создание валидационного набора данных (например, 20% от тренировочного)
val_size = int(0.2 * len(train_dataset))
train_size = len(train_dataset) - val_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Модель VGG
class ImprovedVGG(nn.Module):
    def __init__(self, num_classes=10):
        super(ImprovedVGG, self).__init__()
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Block 2
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Block 3
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.classifier = nn.Sequential(
            nn.Linear(128 * 28 * 28, 512),  # Adjusted for the output size after feature extraction
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(256, num_classes)  # Output layer for the number of classes
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.classifier(x)
        return x

# Инициализация модели
model = ImprovedVGG(num_classes=num_classes).to(device)
summary(model, (3, 224, 224))  # Измените размер входа на (3, 224, 224)

# Оптимизатор и функция потерь
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

# Планировщик
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

num_epochs = 10

# Обучение модели
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    all_labels = []
    all_preds = []
    correct = 0
    total = 0

    start_time = time.time()

    for images, labels in tqdm(train_loader, desc=f"Training Epoch {epoch + 1}/{num_epochs}"):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    end_time = time.time()
    epoch_time = end_time - start_time

    epoch_loss = running_loss / len(train_loader)
    accuracy = correct / total
    f1 = f1_score(all_labels, all_preds, average='weighted')

    print(f"Training Epoch [{epoch + 1}/{num_epochs}], "
          f"Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}, Time: {epoch_time:.2f}s")

    # Валидация
    model.eval()
    val_running_loss = 0.0
    val_all_labels = []
    val_all_preds = []
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for val_images, val_labels in tqdm(val_loader, desc=f"Validation Epoch {epoch + 1}/{num_epochs}"):
            val_images = val_images.to(device)
            val_labels = val_labels.to(device)

            val_outputs = model(val_images)
            val_loss = criterion(val_outputs, val_labels)

            val_running_loss += val_loss.item()

            _, val_predicted = torch.max(val_outputs.data, 1)
            val_all_labels.extend(val_labels.cpu().numpy())
            val_all_preds.extend(val_predicted.cpu().numpy())

            val_total += val_labels.size(0)
            val_correct += (val_predicted == val_labels).sum().item()

    val_epoch_loss = val_running_loss / len(val_loader)
    val_accuracy = val_correct / val_total
    val_f1 = f1_score(val_all_labels, val_all_preds, average='weighted')

    print(f"Validation Epoch [{epoch + 1}/{num_epochs}], "
          f"Loss: {val_epoch_loss:.4f}, Accuracy: {val_accuracy:.4f}, F1 Score: {val_f1:.4f}")

    # Обновление планировщика
    scheduler.step()

    # Сохранение чекпоинта каждые 5 эпох
    if (epoch + 1) % 5 == 0:
        checkpoint_path = f"checkpoint_epoch_{epoch + 1}.pth"
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': epoch_loss,
        }, checkpoint_path)
        print(f"Checkpoint saved: {checkpoint_path}")

print("Training complete!")

# Функция для получения результатов
def get_result(model: torch.nn.Module, transform: transforms.Compose, root_dir_test: str):
    dataset = CustomTestDataset(root_dir=root_dir_test, transform=transform)
    dl = DataLoader(dataset, batch_size=128)

    model.eval()
    ans = []

    for img, label in tqdm(dl):
        img = img.to(device)
        label = label.to(device)
        pred = model(img)
        preds = torch.argmax(pred, dim=1)

        res = torch.cat((label.unsqueeze(1), preds.unsqueeze(1)), dim=1)
        ans.extend(res.cpu())

    if not ans:
        print("Error: ans array is empty. Check the prediction process.")
        return

    ans = [[element.item() for element in row] for row in ans]

    output_file = 'result.csv'

    try:
        with open(output_file, 'w', newline="") as out_file:
            writer = csv.writer(out_file, delimiter=',')
            writer.writerow(['id', 'target_feature'])
            writer.writerows(ans)
        print("Results saved to", output_file)
    except Exception as e:
        print(f"Error saving results: {e}")

# Вызов функции для тестирования
get_result(model, transform, root_dir_test)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 224, 224]             896
              ReLU-2         [-1, 32, 224, 224]               0
       BatchNorm2d-3         [-1, 32, 224, 224]              64
            Conv2d-4         [-1, 32, 224, 224]           9,248
              ReLU-5         [-1, 32, 224, 224]               0
       BatchNorm2d-6         [-1, 32, 224, 224]              64
         MaxPool2d-7         [-1, 32, 112, 112]               0
            Conv2d-8         [-1, 64, 112, 112]          18,496
              ReLU-9         [-1, 64, 112, 112]               0
      BatchNorm2d-10         [-1, 64, 112, 112]             128
           Conv2d-11         [-1, 64, 112, 112]          36,928
             ReLU-12         [-1, 64, 112, 112]               0
      BatchNorm2d-13         [-1, 64, 112, 112]             128
        MaxPool2d-14           [-1, 64,

Training Epoch 1/10:   0%|                                                                                                                              | 0/387 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Kirill\\PycharmProjects\\nto\\img_train\\6652'

In [3]:

# Установка устройства
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Пути к данным
source_directory = r'D:\data sets\ml-intensive-yandex-academy-autumn-2024\human_poses_data'
csv_train_ans = r'D:\data sets\ml-intensive-yandex-academy-autumn-2024\human_poses_data\train_answers.csv'
root_dir_train = r'D:\data sets\ml-intensive-yandex-academy-autumn-2024\human_poses_data\img_train'
root_dir_test = r'D:\data sets\ml-intensive-yandex-academy-autumn-2024\human_poses_data\img_test'
activity_categories_file = r'activity_categories.csv'

# Загрузка категорий активности
activity_categories = pd.read_csv(activity_categories_file)
activity_labels = activity_categories['category'].tolist()

# Загрузка данных
train_annotations = pd.read_csv(csv_train_ans)

# Преобразуем первый столбец в строки, если это необходимо
train_annotations.iloc[:, 0] = train_annotations.iloc[:, 0].astype(str)

# Проверяем структуру данных
print(train_annotations.head())

class CustomDataset(Dataset):
    def __init__(self, img_dir, annotations, transform=None):
        self.img_dir = img_dir
        self.annotations = annotations
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.annotations.iloc[idx, 0])
        if not os.path.exists(img_path):
            print(f"Файл не найден: {img_path}")
            return None, None  # Или выбросьте исключение

        image = Image.open(img_path).convert("RGB")
        label = self.annotations.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)

        return image, label

# Загрузка данных
train_annotations = pd.read_csv(csv_train_ans)

# Приведение типов
train_annotations['img_id'] = train_annotations['img_id'].astype(str)

# Проверяем структуру данных
print(train_annotations.head())

# Загрузка обучающего набора данных
train_dataset = CustomDataset(root_dir_train, train_annotations, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Трансформации
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Загрузка обучающего и тестового наборов данных
train_dataset = CustomDataset(root_dir_train, train_annotations, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Создание модели VGG
model = models.vgg16(weights='DEFAULT')  # Используем 'DEFAULT' для загрузки предобученных весов
num_classes = len(activity_labels)

# Изменяем последний слой
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
model = model.to(device)

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Обучение модели
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    total = 0
    correct = 0

    for images, labels in tqdm(train_loader, desc=f"Training Epoch {epoch + 1}/{num_epochs}"):
        if images is None or labels is None:
            continue  # Пропускаем итерацию, если изображение или метка отсутствуют
        images = images.to(device)
        labels = labels.to(device)

        # Обнуление градиентов
        optimizer.zero_grad()

        # Прямой проход
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Обратный проход и оптимизация
        loss.backward()
        optimizer.step()

        # Обновление потерь
        running_loss += loss.item() * images.size(0)
        total += labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum().item()

    # Вычисление среднего лосса за эпоху
    epoch_loss = running_loss / len(train_loader.dataset)
    accuracy = correct / total
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}')

# Сохранение модели
torch.save(model.state_dict(), 'vgg16_finetuned.pth')
print("Модель сохранена как 'vgg16_finetuned.pth'")

C:\Users\Kirill\AppData\Local\Temp\1\ipykernel_23856\2245471223.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0        17363
1         8612
2         8244
3         9264
4         5382
         ...  
12362    11503
12363    12201
12364     5489
12365      883
12366    16116
Name: img_id, Length: 12367, dtype: object' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  train_annotations.iloc[:, 0] = train_annotations.iloc[:, 0].astype(str)


  img_id  target_feature
0  17363               2
1   8612               5
2   8244               0
3   9264               0
4   5382               6
  img_id  target_feature
0  17363               2
1   8612               5
2   8244               0
3   9264               0
4   5382               6


Training Epoch 1/10:   0%|                                                                                                                              | 0/387 [00:00<?, ?it/s]

Файл не найден: D:\data sets\ml-intensive-yandex-academy-autumn-2024\human_poses_data\img_train\824
Файл не найден: D:\data sets\ml-intensive-yandex-academy-autumn-2024\human_poses_data\img_train\15030
Файл не найден: D:\data sets\ml-intensive-yandex-academy-autumn-2024\human_poses_data\img_train\14681
Файл не найден: D:\data sets\ml-intensive-yandex-academy-autumn-2024\human_poses_data\img_train\8017
Файл не найден: D:\data sets\ml-intensive-yandex-academy-autumn-2024\human_poses_data\img_train\16034
Файл не найден: D:\data sets\ml-intensive-yandex-academy-autumn-2024\human_poses_data\img_train\12081
Файл не найден: D:\data sets\ml-intensive-yandex-academy-autumn-2024\human_poses_data\img_train\10236
Файл не найден: D:\data sets\ml-intensive-yandex-academy-autumn-2024\human_poses_data\img_train\98
Файл не найден: D:\data sets\ml-intensive-yandex-academy-autumn-2024\human_poses_data\img_train\7523
Файл не найден: D:\data sets\ml-intensive-yandex-academy-autumn-2024\human_poses_data\img

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'NoneType'>

In [18]:
activity_categories = pd.read_csv(activity_categories_file)
print(activity_categories.columns)  # Выводим имена столбцов
activity_labels = activity_categories['category'].tolist()

Index(['id', 'category'], dtype='object')


In [7]:
def __getitem__(self, idx):
    img_path = os.path.join(self.img_dir, self.annotations.iloc[idx, 0])
    if not os.path.exists(img_path):
        print(f"Файл не найден: {img_path}")  # Выводим сообщение, если файл не найден
        # Вы можете вернуть пустое изображение или выбросить исключение
        return None, None
    image = Image.open(img_path).convert("RGB")
    label = self.annotations.iloc[idx, 1]

    if self.transform:
        image = self.transform(image)

    return image, label

In [24]:
print(train_annotations.head())

  img_id  target_feature
0  17363               2
1   8612               5
2   8244               0
3   9264               0
4   5382               6


In [8]:
class CustomDataset(Dataset):
    def __init__(self, img_dir, annotations, transform=None):
        self.img_dir = img_dir
        self.annotations = annotations
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.annotations.iloc[idx, 0])
        if not os.path.exists(img_path):
            print(f"Файл не найден: {img_path}")
            return None, None  # Или выбросьте исключение

        image = Image.open(img_path).convert("RGB")
        label = self.annotations.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)

        return image, label


In [3]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  # Используем CUDA или CPU

print("Для обучения выбран девайс {}".format(device))

Для обучения выбран девайс cuda:0


In [24]:
import os
import pandas as pd
from PIL import Image
from torchvision import transforms



csv_train_ans = pd.read_csv('train_answers.csv')

# Проверка типа переменной
print(type(csv_train_ans))  # Должно быть <class 'pandas.core.frame.DataFrame'>


X = []
y = []

# Итерация по строкам DataFrame
for index, row in csv_train_ans.iterrows():
    image_id = row['img_id']  # Используйте имя столбца
    action = row['target_feature']  # Используйте имя столбца
    
    image_path = os.path.join('D:\\data sets\\ml-intensive-yandex-academy-autumn-2024\\human_poses_data', 'D:\\data sets\\ml-intensive-yandex-academy-autumn-2024\\human_poses_data\\img_train', f'{image_id}.jpg')
    X.append(image_path)
    y.append(action)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ColorJitter(brightness=1.5),  # Уменьшение яркости
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Применение трансформаций к первому изображению
image = Image.open(X[0])  # Открываем изображение
transformed_image = transform(image)  # Применяем трансформации
print(transformed_image.shape)

<class 'pandas.core.frame.DataFrame'>
torch.Size([3, 224, 224])


In [29]:
source_directory = r'D:\data sets\ml-intensive-yandex-academy-autumn-2024\human_poses_data'
csv_train_ans = r'D:\data sets\ml-intensive-yandex-academy-autumn-2024\human_poses_data\train_answers.csv'
root_dir_train = r'D:\data sets\ml-intensive-yandex-academy-autumn-2024\human_poses_data\img_train'
root_dir_test = r'D:\data sets\ml-intensive-yandex-academy-autumn-2024\human_poses_data\img_test'
activity_categories_file = r'activity_categories.csv'

# Преобразования для данных
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ColorJitter(brightness=1.5),  # Уменьшение яркости
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

initial_trans = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Загрузка категорий
activity_categories = pd.read_csv(activity_categories_file)
category_map = dict(zip(activity_categories['id'], activity_categories['category']))

# Класс для пользовательского датасета
class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, str(self.data_frame.iloc[idx, 0]) + ".jpg")  # ID изображений в первом столбце
        image = Image.open(img_name).convert('RGB')
        label = self.data_frame.iloc[idx, 1]  # Метки во втором столбце

        if self.transform:
            image = self.transform(image)

        return image, label

# Загрузка данных
train_dataset = CustomDataset(csv_file=csv_train_ans, root_dir=root_dir_train, transform=transform)
train_loader = DataLoader(dataset=train_dataset, batch_size=128, shuffle=True)

train_indices, val_indices = train_test_split(range(len(train_dataset)), test_size=0.2, random_state=42)

train_subset = Subset(train_dataset, train_indices)
val_subset = Subset(train_dataset, val_indices)

# Создание загрузчиков для обучающего и валидационного наборов
train_loader = DataLoader(dataset=train_subset, batch_size=128, shuffle=True)
val_loader = DataLoader(dataset=val_subset, batch_size=128, shuffle=False)

class ImprovedVGG(nn.Module):
    def __init__(self, num_classes=20):
        super(ImprovedVGG, self).__init__()
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Block 2
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Block 3
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Измените размер в соответствии с выходом после сверток
        self.classifier = nn.Sequential(
            nn.Linear(128 * 28 * 28, 512),  # Убедитесь, что размер соответствует выходу
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(256, num_classes)  # Выходной слой для количества классов
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)  # Преобразование тензора в одномерный
        x = self.classifier(x)
        return x

# Инициализация модели
num_classes = len(activity_categories)  # Количество классов в вашем датасете
model = ImprovedVGG(num_classes=num_classes).to(device)   # Количество классов в вашем датасете
# model = ResNet18(num_classes=num_classes)

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-8, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)

# Обучение модели
num_epochs = 20  # Установите количество эпох
losses = []
accuracies = []
f1_scores = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    all_labels = []
    all_preds = []
    correct = 0
    total = 0

    start_time = time.time()  # Запомнить время начала эпохи

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        images, labels = images.to(device), labels.to(device)  # Перемещение данных на устройство

        # Обнуляем градиенты
        optimizer.zero_grad()

        # Прямой проход
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Обратный проход и оптимизация
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Сохранение меток и предсказаний для F1 Score
        _, predicted = torch.max(outputs.data, 1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

        # Обновление correct и total
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    end_time = time.time()  # Запомнить время конца эпохи
    epoch_time = end_time - start_time  # Вычислить время эпохи

    # Вычисление среднего лосса за эпоху
    epoch_loss = running_loss / len(train_loader)
    losses.append(epoch_loss)

    # Вычисление точности и F1 Score
    accuracy = correct / total
    accuracies.append(accuracy)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    f1_scores.append(f1)

    # Валидация
    model.eval()
    val_running_loss = 0.0
    val_correct = 0
    val_total = 0
    val_all_labels = []
    val_all_preds = []

    with torch.no_grad():  # Отключаем градиенты для валидации
        for val_images, val_labels in val_loader:
            val_images, val_labels = val_images.to(device), val_labels.to(device)

            val_outputs = model(val_images)
            val_loss = criterion(val_outputs, val_labels)

            val_running_loss += val_loss.item()

            # Сохранение меток и предсказаний для F1 Score
            _, val_predicted = torch.max(val_outputs.data, 1)
            val_all_labels.extend(val_labels.cpu().numpy())
            val_all_preds.extend(val_predicted.cpu().numpy())

            # Обновление val_correct и val_total
            val_total += val_labels.size(0)
            val_correct += (val_predicted == val_labels).sum().item()

    # Вычисление валидационной потери и точности
    val_epoch_loss = val_running_loss / len(val_loader)
    val_accuracy = val_correct / val_total

    # Вывод результатов для текущей эпохи
    print(f"Epoch [{epoch + 1}/{num_epochs}], "
          f"Train Loss: {epoch_loss:.4f}, Train Accuracy: {accuracy:.4f}, Train F1 Score: {f1:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

    # Сохранение чекпоинта каждые 5 эпох
    if (epoch + 1) % 5 == 0:
        checkpoint_path = f"checkpoint_epoch_{epoch + 1}.pth"
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': epoch_loss,
        }, checkpoint_path)
        print(f"Чекпоинт сохранен: {checkpoint_path}")

print("Обучение завершено!")


# Функция для тестирования модели
class CustomTestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.data_frame = os.listdir(root_dir)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data_frame[idx])
        image = Image.open(img_name).convert('RGB')
        label = self.data_frame[idx].replace(".jpg", '')  # Извлечение ID из имени файла

        if self.transform:
            image = self.transform(image)

        return image, int(label)

def get_result(model: torch.nn.Module, transform: transforms.Compose, root_dir_test: str, category_map: dict):
    # Создание тестового датасета и загрузчика
    dataset = CustomTestDataset(root_dir=root_dir_test, transform=transform)
    dl = DataLoader(dataset, batch_size=128)

    model.eval()  # Установка модели в режим оценки
    ans = []

    # Перебор данных в загрузчике
    for img, label in tqdm(dl):
        img = img.to(device)
        label = label.to(device)  # Перемещение изображений на устройство
        pred = model(img)  # Предсказания модели
        preds = torch.argmax(pred, dim=1)

        # Конкатенация меток и предсказаний
        res = torch.cat((label.unsqueeze(1), preds.unsqueeze(1)), dim=1)
        ans.extend(res.cpu())
    
    # Проверка, есть ли данные в ans
    if not ans:
        print("Ошибка: массив ans пуст. Проверьте процесс предсказания.")
        return

    # Преобразование ans в список
    ans = [[element.item() for element in row] for row in ans]

    # Создание списка для сохранения результатов с номерами категорий
    results_with_categories = []
    
    for id, pred in ans:
        results_with_categories.append([id, pred])  # Сохраняем ID и номер предсказанной категории

    # Убедитесь, что файл будет создан в текущей директории
    output_file = 'result.csv'  # Сохраняем в текущей директории

    # Запись результатов в CSV файл
    try:
        with open(output_file, 'w', newline="") as out_file:
            writer = csv.writer(out_file, delimiter=',')
            writer.writerow(['id', 'target_feature'])  # Заголовки
            writer.writerows(results_with_categories)  # Запись данных
        print("Результаты успешно сохранены в", output_file)
    except Exception as e:
        print(f"Ошибка при записи в файл: {e}")

# Вызов функции для получения результатов на тестовых данных
get_result(model, transform, root_dir_test, category_map)

Epoch 1/20:   4%|████▉                                                                                                                           | 3/78 [00:31<12:57, 10.36s/it]


KeyboardInterrupt: 

In [ ]:
from PIL import Image
import torchvision.transforms as transforms

initial_trans = torchvision.transforms.Compose([
    torchvision.transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

transforms = transforms.Compose([
    transforms.ToPILImage(),  # Convert the tensor back to a PIL image
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomGrayscale(p=0.1),
    transforms.RandomRotation(degrees=30),
    transforms.RandomAffine(degrees=(0, 20), translate=(0.1, 0.1)),
    transforms.GaussianBlur(kernel_size=5, sigma=(0.1, 2.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomPerspective(distortion_scale=0.5, p=0.5),
    transforms.ToTensor(),  # Convert back to tensor after augmentations
    transforms.Normalize((0.5, 0.5, 0.5), (0.25, 0.25, 0.25))  # Normalize the tensor
])

train_answers = pd.read_csv('human_poses_data/train_answers.csv', delimiter=';')

X = []
y = []

for index, row in train_answers.iterrows():
    image_id = row[0]
    action = row[1]
    
    image_path = os.path.join('human_poses_data', 'img_train', f'{image_id}.jpg')
    X.append(image_path)
    y.append(action)

from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def init(self, images, labels):
        self.images = images
        self.labels = labels

    def len(self):
        return len(self.images)

    def getitem(self, idx):
        return initial_trans(Image.open(self.images[idx])), self.labels[idx]

dataset = CustomDataset(X, y)

from torch.utils.data import random_split

train_set, valid_set = random_split(dataset, (0.95, 0.05))

train_loader = DataLoader(train_set, batch_size=24)
valid_loader = DataLoader(valid_set, batch_size=24)
train_loader

from torch.optim.lr_scheduler import ReduceLROnPlateau

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)

def train_and_evaluate(model, train_loader, val_loader, num_epochs): 
    train_losses, val_losses = [], [] 
    val_metrics = [] 

    for epoch in range(num_epochs): 
        model.train() 
        train_loss = 0 
        for images, labels in train_loader: 
            im =  []
            for i in images:
                im.append(transforms(i))
            images, labels = torch.stack(im), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images) 
            loss = criterion(outputs, labels) 
            loss.backward() 
            optimizer.step() 
            train_loss += loss.item() 

        train_loss /= len(train_loader) 
        train_losses.append(train_loss) 
        
        # Validation
        model.eval() 
        val_loss = 0
        all_preds = [] 
        all_labels = [] 
        with torch.no_grad(): 
            for images, labels in val_loader: 
                images, labels = images.to(device), labels.to(device) 
                outputs = model(images)
                loss = criterion(outputs, labels) 
                val_loss += loss.item()
                preds = torch.argmax(outputs, dim=1) 
                all_preds.extend(preds.cpu().numpy()) 
                all_labels.extend(labels.cpu().numpy())

        val_loss /= len(val_loader)
        val_losses.append(val_loss) 
        
        scheduler.step(val_loss)
        acc = accuracy_score(all_labels, all_preds) 
        prec = precision_score(all_labels, all_preds, average="weighted", zero_division=0) 
        rec = recall_score(all_labels, all_preds, average="weighted", zero_division=0) 
        f1 = f1_score(all_labels, all_preds, average="weighted", zero_division=0) 
        val_metrics.append((acc, prec, rec, f1)) 
        
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, "
              f"Val Acc: {acc:.4f}, Val Prec: {prec:.4f}, Val Recall: {rec:.4f}, Val F1: {f1:.4f}") 

train_and_evaluate(model, train_loader, valid_loader, 75)

In [3]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  # Используем CUDA или CPU

print("Для обучения выбран девайс {}".format(device))

Для обучения выбран девайс cuda:0


In [4]:
# Параметры
source_directory = r'D:\data sets\ml-intensive-yandex-academy-autumn-2024\human_poses_data'
csv_train_ans = r'D:\data sets\ml-intensive-yandex-academy-autumn-2024\human_poses_data\train_answers.csv'
root_dir_train = r'D:\data sets\ml-intensive-yandex-academy-autumn-2024\human_poses_data\img_train'
root_dir_test = r'D:\data sets\ml-intensive-yandex-academy-autumn-2024\human_poses_data\img_test'
activity_categories_file = r'activity_categories.csv'

# Преобразования для данных
transform = transforms.Compose([
    transforms.Resize(224, 224),
    transforms.RandomRotation(3),
    transforms.ColorJitter(brightness=1.5, contrast=1, saturation=0.2, hue=1), 
    transforms.RandomApply([transforms.RandomAffine(degrees=2, translate=(0.1, 0.1))]),
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=1, sigma=(0.1, 2)), 
    transforms.RandomGrayscale()]), transforms.RandomHorizontalFlip(),
    transforms.RandomChoice([transforms.RandomAdjustSharpness(1), 
    transforms.RandAugment()]), transforms.ToTensor(),
    ToDtype(torch.float32, scale=True),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

initial_trans = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Загрузка категорий
activity_categories = pd.read_csv(activity_categories_file)
category_map = dict(zip(activity_categories['id'], activity_categories['category']))

# Класс для пользовательского датасета
class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, str(self.data_frame.iloc[idx, 0]) + ".jpg")  # ID изображений в первом столбце
        image = Image.open(img_name).convert('RGB')
        label = self.data_frame.iloc[idx, 1]  # Метки во втором столбце

        if self.transform:
            image = self.transform(image)

        return image, label

# Загрузка данных
train_dataset = CustomDataset(csv_file=csv_train_ans, root_dir=root_dir_train, transform=transform)
train_indices, val_indices = train_test_split(range(len(train_dataset)), test_size=0.2, random_state=42)

train_subset = Subset(train_dataset, train_indices)
val_subset = Subset(train_dataset, val_indices)

# Создание загрузчиков для обучающего и валидационного наборов
train_loader = DataLoader(dataset=train_subset, batch_size=256, shuffle=True)
val_loader = DataLoader(dataset=val_subset, batch_size=256, shuffle=False)

# Класс для валидационного датасета с использованием initial_trans
class ValidationDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, str(self.data_frame.iloc[idx, 0]) + ".jpg")  # ID изображений в первом столбце
        image = Image.open(img_name).convert('RGB')
        label = self.data_frame.iloc[idx, 1]  # Метки во втором столбце

        if self.transform:
            image = self.transform(image)

        return image, label

# Создание валидационного загрузчика с использованием initial_trans
val_dataset = ValidationDataset(csv_file=csv_train_ans, root_dir=root_dir_train, transform=initial_trans)
val_loader = DataLoader(dataset=val_dataset, batch_size=256, shuffle=False)

# Определение модели, функции потерь и оптимизатора
class ImprovedVGG(nn.Module):
    def __init__(self, num_classes=20):
        super(ImprovedVGG, self).__init__()
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Block 2
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Block 3
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Измените размер в соответствии с выходом после сверток
        self.classifier = nn.Sequential(
            nn.Linear(128 * 28 * 28, 512),  # Убедитесь, что размер соответствует выходу
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(256, num_classes)  # Выходной слой для количества классов
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)  # Преобразование тензора в одномерный
        x = self.classifier(x)
        return x

# Инициализация модели
num_classes = len(activity_categories)  # Количество классов в вашем датасете
model = ImprovedVGG(num_classes=num_classes).to(device)  # Перемещение модели на устройство

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-8, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)

# Обучение модели
num_epochs = 44  # Установите количество эпох
losses = []
accuracies = []
f1_scores = []

# Обучение модели
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    all_labels = []
    all_preds = []
    correct = 0
    total = 0

    start_time = time.time()  # Запомнить время начала эпохи

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        images, labels = images.to(device), labels.to(device)

        # Обнуляем градиенты
        optimizer.zero_grad()

        # Прямой проход
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Обратный проход и оптимизация
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Сохранение меток и предсказаний для F1 Score
        _, predicted = torch.max(outputs.data, 1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

        # Обновление correct и total
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    end_time = time.time()  # Запомнить время конца эпохи
    epoch_time = end_time - start_time  # Вычислить время эпохи

    # Вычисление среднего лосса за эпоху
    epoch_loss = running_loss / len(train_loader)
    losses.append(epoch_loss)

    # Вычисление точности и F1 Score
    accuracy = correct / total
    accuracies.append(accuracy)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    f1_scores.append(f1)

    # Валидация
    model.eval()
    val_running_loss = 0.0
    val_correct = 0
    val_total = 0
    val_all_labels = []
    val_all_preds = []

    with torch.no_grad():  # Отключаем градиенты для валидации
        for val_images, val_labels in val_loader:
            val_images, val_labels = val_images.to(device), val_labels.to(device)

            val_outputs = model(val_images)
            val_loss = criterion(val_outputs, val_labels)

            val_running_loss += val_loss.item()

            # Сохранение меток и предсказаний для F1 Score
            _, val_predicted = torch.max(val_outputs.data, 1)
            val_all_labels.extend(val_labels.cpu().numpy())
            val_all_preds.extend(val_predicted.cpu().numpy())

            # Обновление val_correct и val_total
            val_total += val_labels.size(0)
            val_correct += (val_predicted == val_labels).sum().item()

    # Вычисление валидационной потери и точности
    val_epoch_loss = val_running_loss / len(val_loader)
    val_accuracy = val_correct / val_total

    # Вывод результатов для текущей эпохи
    print(f"Epoch [{epoch + 1}/{num_epochs}], "
          f"Train Loss: {epoch_loss:.4f}, Train Accuracy: {accuracy:.4f}, Train F1 Score: {f1:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

    # Сохранение чекпоинта каждые 5 эпох
    if (epoch + 1) % 5 == 0:
        checkpoint_path = f"checkpoint_epoch_{epoch + 1}.pth"
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': epoch_loss,
        }, checkpoint_path)
        print(f"Чекпоинт сохранен: {checkpoint_path}")

print("Обучение завершено!")

# Функция для тестирования модели
class CustomTestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.data_frame = os.listdir(root_dir)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data_frame[idx])
        image = Image.open(img_name).convert('RGB')
        label = self.data_frame[idx].replace(".jpg", '')  # Извлечение ID из имени файла

        if self.transform:
            image = self.transform(image)

        return image, int(label)

def get_result(model: torch.nn.Module, transform: transforms.Compose, root_dir_test: str, category_map: dict):
    # Создание тестового датасета и загрузчика
    dataset = CustomTestDataset(root_dir=root_dir_test, transform=transform)
    dl = DataLoader(dataset, batch_size=128)

    model.eval()  # Установка модели в режим оценки
    ans = []

    # Перебор данных в загрузчике
    for img, label in tqdm(dl):
        img = img.to(device)
        label = label.to(device)  # Перемещение изображений на устройство
        pred = model(img)  # Предсказания модели
        preds = torch.argmax(pred, dim=1)

        # Конкатенация меток и предсказаний
        res = torch.cat((label.unsqueeze(1), preds.unsqueeze(1)), dim=1)
        ans.extend(res.cpu())
    
    # Проверка, есть ли данные в ans
    if not ans:
        print("Ошибка: массив ans пуст. Проверьте процесс предсказания.")
        return

    # Преобразование ans в список
    ans = [[element.item() for element in row] for row in ans]

    # Создание списка для сохранения результатов с номерами категорий
    results_with_categories = []
    
    for id, pred in ans:
        results_with_categories.append([id, pred])  # Сохраняем ID и номер предсказанной категории

    # Убедитесь, что файл будет создан в текущей директории
    output_file = 'result.csv'  # Сохраняем в текущей директории

    # Запись результатов в CSV файл
    try:
        with open(output_file, 'w', newline="") as out_file:
            writer = csv.writer(out_file, delimiter=',')
            writer.writerow(['id', 'target_feature'])  # Заголовки
            writer.writerows(results_with_categories)  # Запись данных
        print("Результаты успешно сохранены в", output_file)
    except Exception as e:
        print(f"Ошибка при записи в файл: {e}")

# Вызов функции для получения результатов на тестовых данных
get_result(model, transform, root_dir_test, category_map)

Epoch 1/44: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 49/49 [17:58<00:00, 22.00s/it]


Epoch [1/44], Train Loss: 4.3650, Train Accuracy: 0.1529, Train F1 Score: 0.1479


Epoch 2/44: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 49/49 [15:10<00:00, 18.57s/it]


Epoch [2/44], Train Loss: 2.5993, Train Accuracy: 0.2114, Train F1 Score: 0.1798


Epoch 3/44:  41%|███████████████████████████████████████████████████▊                                                                           | 20/49 [04:55<07:08, 14.77s/it]


KeyboardInterrupt: 